In [ ]:
import uproot as up
import numpy as np
import time
import iminuit
from iminuit import Minuit
from iminuit.cost import ExtendedUnbinnedNLL
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import matplotlib.pyplot as plt
import sys
import tensorflow as tf
tf.device('GPU:4')


In [ ]:
sys.path.append('/software/pc24403/tfpcbpggsz/func')
sys.path.append('/software/pc24403/tfpcbpggsz/amp')
from amp import *


mc_path = '/shared/scratch/pc24403/amp'

amp_Data_dk_dd_p, ampbar_Data_dk_dd_p = getAmp('b2dk_DD', 'p')

amp_dk_dd_p = np.load(mc_path + '/Int_b2dk_DD_p_amp.npy')
ampbar_dk_dd_p = np.load(mc_path + '/Int_b2dk_DD_p_ampbar.npy')


# Illustration on the amplitude

In [ ]:
s12, s13 = getMass('b2dk_DD_p', 'int')

In [ ]:
def DeltadeltaD(A, Abar):
    var = np.angle(A*np.conj(Abar)) 

    return var

In [ ]:
#plt.scatter(s12, s13, cmap='rainbow',s=0.1, c=DeltadeltaD(amp_dk_dd_p, ampbar_dk_dd_p).flatten())
#plt.colorbar()

In [ ]:
def totalAmplitudeSquared_XY(Bsign=1, amp=[], ampbar=[], x=(0,0,0,0)):

    phase = DeltadeltaD(amp, ampbar)
    absA = tf.abs(amp)
    absAbar = tf.abs(ampbar)


    if Bsign == 1:
        xPlus = x[0]
        yPlus = x[1]
        rB2 = xPlus**2 + yPlus**2
        return (absA **2 * rB2 + absAbar**2 + 2 * (absA * absAbar) * (xPlus * tf.cos(phase) - yPlus * tf.sin(phase)))
    
    elif Bsign == -1:
        xMinus = x[2]
        yMinus = x[3]
        rB2 = xMinus**2 + yMinus**2

        return (absA**2  + absAbar **2 * rB2 + 2 * (absA * absAbar) * (xMinus * tf.cos(phase) + yMinus * tf.sin(phase)))

In [ ]:
x = (-0.0897, -0.011, 0.0586, 0.0688)

prob = totalAmplitudeSquared_XY(1, ampbar_dk_dd_p, amp_dk_dd_p, x)
prob = prob.numpy().flatten()


In [ ]:
ampbar_dk_dd_p[:1,1:]


In [ ]:
amp_dk_dd_p[:2,1:]

In [ ]:
s12_data , s13_data = getMass('b2dk_DD', 'p')
scale_factor = np.zeros(len(s12))
scale_factor = scale_factor + 1

In [ ]:
fig, (ax0, ax1, ax2) = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(24, 6))
ax0.scatter(s12_data, s13_data,s=0.1)
ax0.set_xlabel('$s_-$')
ax0.set_ylabel('$s_+$')

nbins = 100

bin_edges = np.linspace(0.3, 3.2, nbins+1)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
# Assuming s1 is sorted and binned accordingly, this is a placeholder for counting entries per bin
counts, _ = np.histogram(s12_data, bins=bin_edges)
# Calculate the standard deviation for the error bars
std_devs = np.sqrt(counts)
ax1.errorbar(bin_centers, counts, yerr=std_devs, fmt='+', color='darkviolet', label='Toy Data')
ax1.hist(s12, bins=100, histtype='step', color='dodgerblue',weights=prob/np.mean(prob)/len(s12)*len(s12_data))
ax1.set_xlabel('$s_-$')

bin_edges = np.linspace(0.3, 3.2, nbins+1)
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
# Assuming s1 is sorted and binned accordingly, this is a placeholder for counting entries per bin
counts, _ = np.histogram(s13_data, bins=bin_edges)
# Calculate the standard deviation for the error bars
std_devs = np.sqrt(counts)
ax2.errorbar(bin_centers, counts, yerr=std_devs, fmt='+', color='darkviolet', label='Toy Data')
ax2.hist(s13, bins=100, histtype='step', color='dodgerblue',weights=prob/np.mean(prob)/len(s13)*len(s12_data))

plt.show()
